# Reconciliation #03: Insurance Layer Allocation

## Overview
This notebook verifies that loss events are correctly allocated across a
multi-layer insurance program. Specifically it checks:

- The retention is absorbed correctly before any layer responds.
- Each layer pays within its attachment point and per-occurrence limit.
- Aggregate limits deplete correctly across multiple claims.
- **No dollars appear or disappear** -- the conservation equation holds for
  every claim:
  `Retention + Sum(Layer Recoveries) + Uninsured Excess == Ground-Up Loss`

## Prerequisites
- `ergodic_insurance` package installed
- `_reconciliation_helpers.py` in the same directory

## Estimated runtime
< 10 seconds

## Audience
Actuaries, developers, and QA engineers validating the insurance allocation engine.

In [ ]:
"""Setup: imports, helpers, and reproducibility."""
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings("ignore")

# --- reconciliation helpers ------------------------------------------------
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
# Also add the reconciliation directory itself
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath(".")), "ergodic_insurance", "notebooks", "reconciliation"))
# And the notebook's own directory
sys.path.insert(0, os.path.abspath(os.path.dirname("_reconciliation_helpers.py")))

from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df
)

# --- insurance APIs -------------------------------------------------------
from ergodic_insurance.insurance_program import (
    InsuranceProgram, EnhancedInsuranceLayer
)

# --- reproducibility -------------------------------------------------------
np.random.seed(42)

print("Setup complete.")

In [ ]:
notebook_header(
    3,
    "Insurance Layer Allocation",
    "Verify that loss events are correctly allocated across a multi-layer "
    "insurance program -- retention, per-occurrence limits, aggregate depletion, "
    "and dollar conservation."
)

## 1. Layer Configuration

We build a 3-layer program on top of a $50K self-insured retention:

| Layer | Attachment | Limit | Covers |
|-------|-----------|-------|--------|
| Retention | $0 | $50K | $0 -- $50K |
| Layer 1 (Primary) | $50K | $50K | $50K -- $100K |
| Layer 2 (1st Excess) | $100K | $200K | $100K -- $300K |
| Layer 3 (Umbrella) | $300K | $500K | $300K -- $800K |

Any loss above $800K is **uninsured excess**.

In [ ]:
section_header("1. Layer Configuration")

RETENTION = 50_000

layer_1 = EnhancedInsuranceLayer(
    attachment_point=50_000,
    limit=50_000,
    base_premium_rate=0.05,
)
layer_2 = EnhancedInsuranceLayer(
    attachment_point=100_000,
    limit=200_000,
    base_premium_rate=0.03,
)
layer_3 = EnhancedInsuranceLayer(
    attachment_point=300_000,
    limit=500_000,
    base_premium_rate=0.01,
)

program = InsuranceProgram(
    layers=[layer_1, layer_2, layer_3],
    deductible=RETENTION,
    name="Reconciliation Test Program",
)

# Display the program summary
summary = program.get_program_summary()
print(f"Program: {summary['program_name']}")
print(f"Deductible (SIR): {fmt_dollar(summary['deductible'])}")
print(f"Total coverage above SIR: {fmt_dollar(summary['total_coverage'])}")
print(f"Maximum covered loss: {fmt_dollar(RETENTION + summary['total_coverage'])}")
print()

layer_df = pd.DataFrame(summary["layers"])
layer_df.columns = ["Attachment", "Limit", "Exhaustion Pt", "Reinstatements", "Premium"]
layer_df.index = ["Layer 1 (Primary)", "Layer 2 (1st Excess)", "Layer 3 (Umbrella)"]
for col in ["Attachment", "Limit", "Exhaustion Pt", "Premium"]:
    layer_df[col] = layer_df[col].apply(lambda v: fmt_dollar(v))
display_df(layer_df, "Program Layers")

## 2. Per-Occurrence Allocation

We process a series of known loss sizes designed to exercise every boundary
in the program:

| Loss | Expected Allocation |
|------|--------------------|
| $25K | Entirely within retention |
| $75K | Retention + partial Layer 1 |
| $150K | Retention + full Layer 1 + partial Layer 2 |
| $400K | Retention + full L1 + full L2 + partial L3 |
| $1M | Retention + full L1 + full L2 + full L3 + $200K uninsured |

In [ ]:
section_header("2. Per-Occurrence Allocation")

checker_allocation = ReconciliationChecker(section="Per-Occurrence Allocation")

# Reset program for clean state
program.reset_annual()

test_losses = [25_000, 75_000, 150_000, 400_000, 1_000_000]
loss_labels = ["$25K", "$75K", "$150K", "$400K", "$1M"]

# Expected allocations (retention, layer_1, layer_2, layer_3, uninsured_excess)
expected = {
    25_000:    {"retention": 25_000,  "L1": 0,      "L2": 0,       "L3": 0,       "uninsured": 0},
    75_000:    {"retention": 50_000,  "L1": 25_000, "L2": 0,       "L3": 0,       "uninsured": 0},
    150_000:   {"retention": 50_000,  "L1": 50_000, "L2": 50_000,  "L3": 0,       "uninsured": 0},
    400_000:   {"retention": 50_000,  "L1": 50_000, "L2": 200_000, "L3": 100_000, "uninsured": 0},
    1_000_000: {"retention": 50_000,  "L1": 50_000, "L2": 200_000, "L3": 500_000, "uninsured": 200_000},
}

allocation_rows = []

with timed_cell("Per-Occurrence Allocation"):
    for loss in test_losses:
        # Process each loss independently -- reset between claims
        program.reset_annual()
        result = program.process_claim(loss)

        # Extract per-layer payments
        layer_payments = {0: 0.0, 1: 0.0, 2: 0.0}
        for lp in result.layers_triggered:
            layer_payments[lp.layer_index] = lp.payment

        total_insurance = result.insurance_recovery
        # Retention is what the insured pays minus the uncovered excess portion
        # In the ClaimResult, deductible_paid = original deductible + uncovered_loss
        retention_actual = min(loss, RETENTION)
        uninsured_actual = result.uncovered_loss

        exp = expected[loss]

        allocation_rows.append({
            "Loss": fmt_dollar(loss),
            "Retention": fmt_dollar(retention_actual),
            "Layer 1": fmt_dollar(layer_payments[0]),
            "Layer 2": fmt_dollar(layer_payments[1]),
            "Layer 3": fmt_dollar(layer_payments[2]),
            "Uninsured": fmt_dollar(uninsured_actual),
            "Total Check": fmt_dollar(retention_actual + total_insurance + uninsured_actual),
        })

        # --- Conservation check ---
        reconstructed = retention_actual + total_insurance + uninsured_actual
        checker_allocation.assert_close(
            reconstructed, loss, tol=0.01,
            message=f"Conservation: {fmt_dollar(loss)} loss",
        )

        # --- Layer-specific checks ---
        checker_allocation.assert_close(
            layer_payments[0], exp["L1"], tol=0.01,
            message=f"Layer 1 payment for {fmt_dollar(loss)} loss",
        )
        checker_allocation.assert_close(
            layer_payments[1], exp["L2"], tol=0.01,
            message=f"Layer 2 payment for {fmt_dollar(loss)} loss",
        )
        checker_allocation.assert_close(
            layer_payments[2], exp["L3"], tol=0.01,
            message=f"Layer 3 payment for {fmt_dollar(loss)} loss",
        )
        checker_allocation.assert_close(
            uninsured_actual, exp["uninsured"], tol=0.01,
            message=f"Uninsured excess for {fmt_dollar(loss)} loss",
        )

alloc_df = pd.DataFrame(allocation_rows)
display_df(alloc_df, "Per-Occurrence Allocation Results")

checker_allocation.display_results()

### Stacked Bar Chart -- Layer Allocation by Loss Size

For each test loss, the chart shows the dollar allocation to each bucket:
retention (grey), Layer 1 (blue), Layer 2 (orange), Layer 3 (green), and
uninsured excess (red).

In [ ]:
"""Stacked bar chart of layer allocation."""

# Re-collect numeric data for charting
chart_data = {"Retention": [], "Layer 1": [], "Layer 2": [], "Layer 3": [], "Uninsured": []}

for loss in test_losses:
    program.reset_annual()
    result = program.process_claim(loss)
    lp = {0: 0.0, 1: 0.0, 2: 0.0}
    for p in result.layers_triggered:
        lp[p.layer_index] = p.payment
    chart_data["Retention"].append(min(loss, RETENTION))
    chart_data["Layer 1"].append(lp[0])
    chart_data["Layer 2"].append(lp[1])
    chart_data["Layer 3"].append(lp[2])
    chart_data["Uninsured"].append(result.uncovered_loss)

colors = ["#999999", "#4a86c8", "#f5a623", "#2ecc71", "#e74c3c"]
categories = list(chart_data.keys())
x = np.arange(len(test_losses))

fig, ax = plt.subplots(figsize=(10, 5))
bottom = np.zeros(len(test_losses))

for cat, color in zip(categories, colors):
    vals = np.array(chart_data[cat])
    ax.bar(x, vals, bottom=bottom, label=cat, color=color, width=0.6)
    bottom += vals

ax.set_xticks(x)
ax.set_xticklabels(loss_labels)
ax.set_xlabel("Ground-Up Loss")
ax.set_ylabel("Dollars")
ax.set_title("Insurance Layer Allocation by Loss Size")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax.legend(loc="upper left")
plt.tight_layout()
plt.show()

## 3. Boundary Testing

Verify behaviour at the exact attachment points and limit exhaustion points
for each layer. These are the most likely places for off-by-one errors.

In [ ]:
section_header("3. Boundary Testing")

checker_boundary = ReconciliationChecker(section="Boundary Tests")

# Boundary losses: at, just below, and just above each critical point
boundary_tests = [
    # (loss, description, expected_L1, expected_L2, expected_L3)
    (50_000,       "At retention (L1 attach)",       0,       0,       0),
    (50_001,       "Just above L1 attach",            1,       0,       0),
    (100_000,      "L1 exhausted / L2 attach",  50_000,       0,       0),
    (100_001,      "Just above L2 attach",      50_000,       1,       0),
    (300_000,      "L2 exhausted / L3 attach",  50_000, 200_000,       0),
    (300_001,      "Just above L3 attach",      50_000, 200_000,       1),
    (800_000,      "L3 exhausted",              50_000, 200_000, 500_000),
    (800_001,      "Just above L3 exhaust",     50_000, 200_000, 500_000),
]

with timed_cell("Boundary Tests"):
    for loss, desc, exp_l1, exp_l2, exp_l3 in boundary_tests:
        program.reset_annual()
        result = program.process_claim(loss)

        lp = {0: 0.0, 1: 0.0, 2: 0.0}
        for p in result.layers_triggered:
            lp[p.layer_index] = p.payment

        # Check each layer
        checker_boundary.assert_close(
            lp[0], exp_l1, tol=0.01,
            message=f"L1 @ {desc}",
        )
        checker_boundary.assert_close(
            lp[1], exp_l2, tol=0.01,
            message=f"L2 @ {desc}",
        )
        checker_boundary.assert_close(
            lp[2], exp_l3, tol=0.01,
            message=f"L3 @ {desc}",
        )

        # Conservation always holds
        retention_actual = min(loss, RETENTION)
        reconstructed = retention_actual + result.insurance_recovery + result.uncovered_loss
        checker_boundary.assert_close(
            reconstructed, loss, tol=0.01,
            message=f"Conservation @ {desc}",
        )

        # Each layer recovery <= its limit
        checker_boundary.check(
            lp[0] <= layer_1.limit + 0.01,
            f"L1 <= limit @ {desc}",
            f"{fmt_dollar(lp[0])} <= {fmt_dollar(layer_1.limit)}",
        )
        checker_boundary.check(
            lp[1] <= layer_2.limit + 0.01,
            f"L2 <= limit @ {desc}",
            f"{fmt_dollar(lp[1])} <= {fmt_dollar(layer_2.limit)}",
        )
        checker_boundary.check(
            lp[2] <= layer_3.limit + 0.01,
            f"L3 <= limit @ {desc}",
            f"{fmt_dollar(lp[2])} <= {fmt_dollar(layer_3.limit)}",
        )

checker_boundary.display_results()

## 4. Aggregate Limit Depletion

We configure Layer 1 as a **hybrid** layer with a $100K aggregate limit
(2x the per-occurrence limit of $50K). After enough claims exhaust
that aggregate, subsequent claims should get **zero** recovery from Layer 1
even though they individually exceed its attachment point.

Test sequence: six $75K claims. Each claim puts $25K into Layer 1
($75K loss minus $50K attachment = $25K, within the $50K per-occurrence cap).

- Claims 1--4: Layer 1 pays $25K each (aggregate used = $25K, $50K, $75K, $100K).
- Claims 5--6: Layer 1 aggregate exhausted ($100K used), pays $0.
- Layer 2 and Layer 3 remain unaffected (per-occurrence, no aggregate).

In [ ]:
section_header("4. Aggregate Limit Depletion")

checker_aggregate = ReconciliationChecker(section="Aggregate Depletion")

# Rebuild Layer 1 with a hybrid limit (per-occurrence $50K, aggregate $100K)
layer_1_agg = EnhancedInsuranceLayer(
    attachment_point=50_000,
    limit=50_000,
    base_premium_rate=0.05,
    limit_type="hybrid",
    per_occurrence_limit=50_000,
    aggregate_limit=100_000,
)

# Layers 2 and 3 remain per-occurrence (no aggregate constraint)
program_agg = InsuranceProgram(
    layers=[layer_1_agg, layer_2, layer_3],
    deductible=RETENTION,
    name="Aggregate Test Program",
)

claim_amount = 75_000  # Each claim: $50K retention + $25K into Layer 1
n_claims = 6

# Each $75K claim puts $25K into L1 (75K - 50K attachment = 25K).
# Aggregate limit is $100K, so 4 claims x $25K = $100K exhausts the aggregate.
# Claims 5 and 6 should get $0 from Layer 1.

agg_rows = []
l1_cumulative = 0.0

with timed_cell("Aggregate Depletion"):
    for i in range(1, n_claims + 1):
        result = program_agg.process_claim(claim_amount)

        lp = {0: 0.0, 1: 0.0, 2: 0.0}
        for p in result.layers_triggered:
            lp[p.layer_index] = p.payment

        l1_cumulative += lp[0]
        retention_actual = min(claim_amount, RETENTION)
        reconstructed = retention_actual + result.insurance_recovery + result.uncovered_loss

        agg_rows.append({
            "Claim #": i,
            "Loss": fmt_dollar(claim_amount),
            "Retention": fmt_dollar(retention_actual),
            "L1 Payment": fmt_dollar(lp[0]),
            "L1 Cumulative": fmt_dollar(l1_cumulative),
            "L2 Payment": fmt_dollar(lp[1]),
            "L3 Payment": fmt_dollar(lp[2]),
            "Uninsured": fmt_dollar(result.uncovered_loss),
        })

        # Conservation check
        checker_aggregate.assert_close(
            reconstructed, claim_amount, tol=0.01,
            message=f"Conservation: claim #{i}",
        )

        # Layer 1 aggregate depletion logic
        if i <= 4:
            # First four claims: each should get $25K from L1
            # (4 x $25K = $100K aggregate)
            checker_aggregate.assert_close(
                lp[0], 25_000, tol=0.01,
                message=f"L1 pays $25K on claim #{i} (aggregate not yet exhausted)",
            )
        else:
            # Claims 5+: L1 aggregate ($100K) is exhausted
            checker_aggregate.check(
                lp[0] <= 0.01,
                f"L1 pays $0 on claim #{i} (aggregate exhausted)",
                f"L1 payment = {fmt_dollar(lp[0])}",
            )

# Verify cumulative L1 does not exceed aggregate
checker_aggregate.check(
    l1_cumulative <= 100_000 + 0.01,
    f"L1 cumulative ({fmt_dollar(l1_cumulative)}) <= aggregate limit ($100,000)",
    f"{fmt_dollar(l1_cumulative)} <= $100,000",
)

# Verify cumulative L1 equals exactly $100K (fully used)
checker_aggregate.assert_close(
    l1_cumulative, 100_000, tol=0.01,
    message="L1 aggregate fully utilized (exactly $100K paid)",
)

agg_df = pd.DataFrame(agg_rows)
display_df(agg_df, "Aggregate Depletion Trace")

checker_aggregate.display_results()

In [ ]:
"""Chart: Layer 1 aggregate depletion over successive claims."""

# Re-run to collect numeric data for chart
program_agg.reset_annual()
l1_payments = []
l1_running = []
running = 0.0

for _ in range(n_claims):
    result = program_agg.process_claim(claim_amount)
    lp = {0: 0.0, 1: 0.0, 2: 0.0}
    for p in result.layers_triggered:
        lp[p.layer_index] = p.payment
    l1_payments.append(lp[0])
    running += lp[0]
    l1_running.append(running)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Left: per-claim Layer 1 payment
claim_nums = list(range(1, n_claims + 1))
bar_colors = ["#4a86c8" if p > 0 else "#cccccc" for p in l1_payments]
ax1.bar(claim_nums, l1_payments, color=bar_colors, edgecolor="#333")
ax1.set_xlabel("Claim #")
ax1.set_ylabel("Layer 1 Payment")
ax1.set_title("Layer 1 Per-Claim Payment")
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax1.set_xticks(claim_nums)

# Right: cumulative Layer 1 vs aggregate limit
ax2.plot(claim_nums, l1_running, marker="o", color="#4a86c8", linewidth=2, label="Cumulative L1")
ax2.axhline(100_000, color="#e74c3c", linestyle="--", linewidth=2, label="Aggregate Limit ($100K)")
ax2.set_xlabel("Claim #")
ax2.set_ylabel("Cumulative Payment")
ax2.set_title("Layer 1 Aggregate Depletion")
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax2.set_xticks(claim_nums)
ax2.legend()

plt.tight_layout()
plt.show()

## Final Summary

Combine all check results across sections and display the overall verdict.

In [ ]:
final_summary(checker_allocation, checker_boundary, checker_aggregate)